## 卷积神经网络简介

#### 加载Keras中的MNIST的数据集

In [36]:
from keras.datasets import mnist
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

In [37]:
print('数据类型：',train_images.dtype)
print('train_images是一个由32位整数组成的3D张量')

数据类型： uint8
train_images是一个由32位整数组成的3D张量


In [38]:
print('训练图像 train_images :',train_images.shape)
print("训练标签 train_labels 前5行:",train_labels[0:5])
print("*******************************************")
print('测试图像 test_images:',test_images.shape)
print("测试标签 test_labels 前5行:",test_labels[0:5])
print('训练图像是60000个矩阵组成的数组，每个矩阵由28*28个整数组成。每个这样的矩阵都是一张灰度图像，元素取值范围为0-32位浮点数')

训练图像 train_images : (60000, 28, 28)
训练标签 train_labels 前5行: [5 0 4 1 9]
*******************************************
测试图像 test_images: (10000, 28, 28)
测试标签 test_labels 前5行: [7 2 1 0 4]
训练图像是60000个矩阵组成的数组，每个矩阵由28*28个整数组成。每个这样的矩阵都是一张灰度图像，元素取值范围为0-32位浮点数


#### 实例化一个小型的卷积神经网络

In [17]:
from keras import layers
from keras import models
models=models.Sequential()
models.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
# 卷积神经网络接收的形状为（image_height,image_width,image_channels）的输入张量（不包括批量维度）
# 通过input_shape=(28,28,1)来完成此设置
models.add(layers.MaxPooling2D((2,2)))
models.add(layers.Conv2D(64,(3,3),activation='relu'))
models.add(layers.MaxPooling2D((2,2)))
models.add(layers.Conv2D(64,(3,3),activation='relu'))

##### 查看卷积神经网络的架构

In [18]:
models.summary()
print('可以看到，每个Conv2D 层和MaxPooling2D 层的输出都是一个形状为(height, width,channels) 的3D 张量。宽度和高度两个维度的尺寸通常会随着网络加深而变小。通道数量由传入Conv2D 层的第一个参数所控制（32 或64）。')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________
可以看到，每个Conv2D 层和MaxPooling2D 层的输出都是一个形状为(height, width,channels) 的3D 张量。宽度和高度两个维度的尺寸通常会随着网络加深而变小。通道数量由传入Conv2D 层的第一个参数所控制（32 或64）。


上述示例中,第一个卷积层接收一个大小为(28,28,1)的特征图，并输出一个大小
为(26,26,32)的特征图，即它在输入上计算32个过滤器。对于这32个输出通道，
每个通道都包含一个26×26的数值网络，它是过滤器对输入的响应图，表示这个过滤器
模式在输入中不同位置的响应。

#### 在卷积神经网络上添加分类器

In [19]:
models.add(layers.Flatten())
models.add(layers.Dense(64,activation='relu'))
models.add(layers.Dense(10,activation='softmax'))
print('将最后的输出张量［大小为(3, 3, 64)］输入到一个密集连接分类器网络中，即Dense 层的堆叠。这些分类器可以处理1D 向量，而当前的输出是3D 张量。首先，我们需要将3D 输出展平为1D，然后在上面添加几个Dense 层。')

将最后的输出张量［大小为(3, 3, 64)］输入到一个密集连接分类器网络中，即Dense 层的堆叠。这些分类器可以处理1D 向量，而当前的输出是3D 张量。首先，我们需要将3D 输出展平为1D，然后在上面添加几个Dense 层。


#### Flatten层用来将输入‘压平’,即把多维的输入一维化，常用在从卷积层到全连接层的过渡。

In [21]:
models.summary()
print('在进入两个Dense 层之前，形状(3, 3, 64) 的输出被展平为形状(576,) 的向量。')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                36928     
__________

In [23]:
from keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
models.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])
models.fit(train_images, train_labels, epochs=5, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 36s 597us/step - loss: 0.1739 - acc: 0.9454
Epoch 2/5
60000/60000 [==============================] - 39s 647us/step - loss: 0.0472 - acc: 0.9851
Epoch 3/5
60000/60000 [==============================] - 41s 675us/step - loss: 0.0320 - acc: 0.9898
Epoch 4/5
60000/60000 [==============================] - 41s 679us/step - loss: 0.0251 - acc: 0.9923
Epoch 5/5
60000/60000 [==============================] - 39s 649us/step - loss: 0.0184 - acc: 0.9941


In [25]:
test_loss, test_acc = models.evaluate(test_images, test_labels)

10000/10000 [==============================] - 2s 211us/step


In [27]:
test_acc

0.9805